In [45]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier



In [2]:
#import data
X_train_full = pd.read_csv("../train.csv")
Test_data = pd.read_csv("../test.csv")

In [9]:
#inspect the data

#X_train_full.head()
X_train_full.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [27]:
# Remove rows with missing target - in this case there are none
# set target and drop target from predictors

y = X_train_full["Survived"]
X_train_full.drop(["Survived"], axis = 1, inplace = True)

In [34]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y, train_size=0.8, test_size=0.2)

In [28]:
#DEAL WITH MISSING DATA
#get names of columns with missing values

col_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]
print(col_with_missing)

['Age', 'Cabin', 'Embarked']


In [35]:
#how many values are missing in each column?
missing_values_count_by_column = X_train_full.isnull().sum()
print(missing_values_count_by_column)

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [36]:
#get numerical and categorical columns
all_columns = X_train_full.columns
print(all_columns)

numerical_columns = [col for col in X_train_full.columns if X_train_full[col].dtype in ["int64", "float64"]]
print("Numerical columns: ", numerical_columns)
categorical_columns = [col for col in X_train_full.columns if X_train_full[col].dtype in ["object"]]
print("Categorical columns: ", categorical_columns)

#check if added columns are all columns
len(all_columns) == len(categorical_columns + numerical_columns)

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Numerical columns:  ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Categorical columns:  ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


True

In [50]:
#check cardinality of categorical columns

low_cardinality_cols = [col for col in X_train_full[categorical_columns] if X_train_full[col].nunique() < 10 and 
                        X_train_full[col].dtype == "object"]
print(low_cardinality_cols)

high_cardinality_cols = [col for col in X_train_full[categorical_columns] if X_train_full[col].nunique() >= 10 and 
                        X_train_full[col].dtype == "object"]

len(categorical_columns) == len(low_cardinality_cols + high_cardinality_cols)

['Sex', 'Embarked']


True

In [41]:
X_train_full[categorical_columns].nunique()

Name        891
Sex           2
Ticket      681
Cabin       147
Embarked      3
dtype: int64

In [59]:
#define numerical and categorical transformers (preprocessing)
#remember to change strategies later to see which one performs best

numerical_transformer = SimpleImputer(strategy = "constant")

low_categorical_transformer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "most_frequent")), 
                                               ("OH_encoder", OneHotEncoder(handle_unknown = "ignore"))])

high_catergorical_transformer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "most_frequent")),
                                                 ("labeler", LabelEncoder())])

#bundle them together in one preprocessor via ColumnTransformer: (syntax like pipeline but don't forget to add columns)

preprocessor = ColumnTransformer(transformers = [("num", numerical_transformer, numerical_columns),
                                                 ("low_cat", low_categorical_transformer, low_cardinality_cols),
                                                 ("high_cat", high_categorical_transformer, high_cardinality_cols)])

In [53]:
#define the model

my_model = XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=6)


In [65]:
#preprocess

for col in all_columns:
    X_train[col] = preprocessor.fit_transform(X_train)
    X_valid[col] = preprocessor.transform(X_valid)


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:192: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return X != X
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:192: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return X != X
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:192: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return X != X
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:192: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  return X != X


AttributeError: 'bool' object has no attribute 'transpose'

In [62]:
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)])

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields Name, Sex, Ticket, Cabin, Embarked